# Pracitice Exercise

## Libraries

In [1]:
from langgraph.graph import START, END, StateGraph, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from IPython.display import Image, display
import os

print("All libraries imported!")

All libraries imported!


## API Key

In [2]:
load_dotenv()
api_key = os.getenv("paid_api")

if not api_key:
    raise ValueError("API Key not found!")
print("API Key loaded successfully")

API Key loaded successfully


## Initialize LLM

In [3]:
llm = ChatOpenAI(
    model = "gpt-4o-mini",
    temperature=0.7,
    api_key=api_key
)

print(f"LLM initialized {llm.model_name}")

LLM initialized gpt-4o-mini


## System Prompt and Assitant

In [4]:
# Sytem prompt that makes the agent act as a helpful customer rep
sys_msg = SystemMessage(
    content = "Your are a polite and helpful customer support rep. Answer customers' questions accurately and be concise."

)

# Sales assitant node
def sales_assist(state: MessagesState) -> dict:
    """
    The assistant node - processes messages and generates response.
    """
    messages = [sys_msg] + state["messages"]

    response = llm.invoke(messages)
    return {"messages": [AIMessage(content=response.content)]}

print("Sales assistant mode defined")



Sales assistant mode defined


## StateGraph with MessagesState

In [5]:
# The graph
builder = StateGraph(MessagesState)
builder.add_node("assistant", sales_assist)

builder.add_edge(START, "assistant")
builder.add_edge("assistant", END)

#Checkpointer for memory
memory = MemorySaver()

agent2 = builder.compile(checkpointer=memory)
print('Agent compiled with memory')

Agent compiled with memory


## Test with mult-turn conversation

In [6]:
def interactive_chat():
    """
    Run an interactive chat session.
    Type 'exit' or 'quit' to stop.
    """
    print("\n" + "="*70)
    print("Interactive Chat Started")
    print("Type your message and press Enter. Type 'exit' to quit.")
    print("="*70 + "\n")

    thread_id = "interactive_session2"

    while True:
        user_input = input("\nYou: ").strip()
        if user_input.lower() in ["exit", "quit"]:
            print("\nGoodbye...")
            break
        
        if not user_input:
            continue
        
        #Get response
        result = agent2.invoke(
            {"messages": [HumanMessage(content=user_input)]},
            config={"configurable": {"thread_id": thread_id}}
        )

        # Print response
        agent_message = result["messages"][-1]
        user_message = result['messages'][-2]
        print(f"\nYou: {user_message.content}")
        print(f"\nAgent: {agent_message.content}")



In [7]:
interactive_chat()


Interactive Chat Started
Type your message and press Enter. Type 'exit' to quit.


You: I bought a phone

Agent: That's great! How can I assist you with your new phone? Do you have any questions or need help with setup?

You: The phone is google pixel 9

Agent: Excellent choice! The Google Pixel 9 has some fantastic features. Do you need assistance with setup, transferring data, or exploring its features?

You: how do i take a screenshot on my phone

Agent: To take a screenshot on your Google Pixel 9, follow these steps:

1. **Press the Power and Volume Down buttons** at the same time and hold them for a moment.
2. You'll see a flash on the screen, and a notification will appear indicating that your screenshot has been taken.

You can find the screenshot in your Photos app or in the notification panel. If you have any other questions, feel free to ask!

Goodbye...
